----------------
Bibliotecas

In [ ]:
# Importando bibliotecas relevantes para o projeto
import numpy as np
import pandas as pd
import seaborn as sns
import math
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support as score, roc_curve, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from collections import Counter

-----------------
Importando o dataframe e explorando os dados

In [ ]:
#Importando o csv com o dataframe
#Corrigido o encode para ANSI
#Feito o parse da coluna 'date' para formato datetime do pandas
dfBase = pd.read_csv('DataSets/full_devices.csv', encoding='ANSI', parse_dates=['date'])
dfBase.info()
dfBase.head()

Nenhum campo vazio

In [ ]:
#Checando campos vazios
dfBase.isnull().sum(axis = 0)

Uma data e device com o registro duplicado

In [ ]:
#Checando par de data e device duplicados
dfBase[dfBase.duplicated(subset=['date','device'])==True]

In [ ]:
#Estatísticas descritivas do df
dfBase.describe()

#attribute 7 e 8 tem exatamente a mesma estatística descritiva, pode ser uma coluna duplicada

Os attribute 7 e attribute 8 são provavelmente atributos iguais duplicados na base, o heatmap ajuda a perceber isso.

Além disso, há uma possível correlação entre o atributte 3 e o 9.

In [ ]:
#Investigando possíveis correlações entre as features
sns.heatmap(dfBase.corr(), annot=True, fmt=".1f", linewidth=.5, cmap='RdBu')

------------------------
Tratamento de dados

- Removendo o registro duplicado
- Removendo a feature duplicada
- Mantive as features 3 e 9. 0.5 não foi correlação suficiente para removê-las.
- Em um cenário real, onde conheço as features e sei o que cada uma representa, poderia ainda incluir aqui um passo de feature engineering

In [ ]:
#Limpando problemas encontrados
dfLimpo = dfBase.drop_duplicates(subset=['date','device']) #Removendo duplicatas
dfLimpo = dfLimpo.drop(columns='attribute8') #Removendo a feature duplicada 'attribute8'
print(dfLimpo.shape)
dfLimpo.head(5)

-------------
Com os dados limpos é hora de separar as variáveis preditores e as de resposta, dividir também nosso dataset em teste e treino com estratificação.

Como os dados estão bem desbalanceados nas classes, utilizei um método de oversampling para balancear melhor, igualando a proporção de falha com não falha.

Escolhi alguns modelos e validei a acurácia deles com uma metodologia de Kfold.

Random_State 42 é a resposta do universo pra tudo. haha


In [ ]:
#Sepração de X e Y e definição do oversample
x = dfLimpo.drop(columns=['date','device','failure'])
y = dfLimpo['failure']
oversample = RandomOverSampler(sampling_strategy='minority')

Regressão logística

In [ ]:
#Regressão logística
modelo = LogisticRegression(random_state=42,class_weight='balanced')

steps = [('over', oversample), ('modelo', modelo)]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, x, y, scoring='f1_micro', cv=cv)
scores.mean()

Árvore de decisão

In [ ]:
#Árvore de decisão
modelo = DecisionTreeClassifier(random_state=42)

steps = [('over', oversample), ('modelo', modelo)]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, x, y, scoring='f1_micro', cv=cv)
scores.mean()

Random Forest

In [ ]:
#Random Forest
modelo = RandomForestClassifier(n_estimators = 100, random_state = 42)

steps = [('over', oversample), ('modelo', modelo)]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, x, y, scoring='f1_micro', cv=cv)
scores.mean()

Naive-Bayes

In [ ]:
#Naive_Bayes
modelo = GaussianNB()

steps = [('over', oversample), ('modelo', modelo)]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, x, y, scoring='f1_micro', cv=cv)
scores.mean()

--------------------------
Tratamento final dos dados para modelo escolhido

In [ ]:
#Sepração de X e Y e definição do oversample
x = dfLimpo.drop(columns=['date','device','failure'])
y = dfLimpo['failure']

oversample = RandomOverSampler(sampling_strategy='minority')
x_over, y_over = oversample.fit_resample(x,y)

x_train, x_test, y_train, y_test = train_test_split(x_over,y_over,train_size=0.7, random_state=42)


print('train: ', x_train.shape, y_train.shape)
print(y_train.value_counts())


In [ ]:
#modelo = LogisticRegression(random_state=42)
#modelo = DecisionTreeClassifier(random_state=42)
modelo = RandomForestClassifier(n_estimators = 100, random_state = 42)
#modelo = GaussianNB()
modelo.fit(x_train, y_train)

y_pred = modelo.predict(x_test)
y_prob = pd.DataFrame(modelo.predict_proba(x_test))

metricas = classification_report(y_test,y_pred)
print(metricas)
print(modelo.score(x_test,y_test))

-------
Matriz de confusão

In [ ]:
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()

In [ ]:
dfPredicts = x_test.copy()
dfPredicts = dfPredicts.reset_index(drop=True)
dfPredicts['y real'] = pd.DataFrame(y_test).reset_index(drop=True)
dfPredicts['y pred'] = pd.DataFrame(y_pred).reset_index(drop=True)
dfPredicts['y prob'] = pd.DataFrame(y_prob[1]).reset_index(drop=True)
dfPredicts